#### Step 1: Combine all load case Excel files

In [1]:
# Import relevant libraries
import pandas as pd
import os

# Folder containing all Excel files
cheeseslab_folder = 'C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\Forces'

# List to store the Excel files
cheeseslab_dataframes = []

# Looping through all Excel files in Cheese Slab Export > Forces folder
for file in os.listdir(cheeseslab_folder):
    if file.endswith('.xlsx') or file.endswith('.xls'):
        file_path = os.path.join(cheeseslab_folder, file)
        # Read the Excel file and append to the list
        cheeseslab_dataframes.append(pd.read_excel(file_path))

# Combine all DataFrames into a single DataFrame
# CS == Cheese slab
all_cs_loadcases = pd.concat(cheeseslab_dataframes, ignore_index=True)

In [8]:
# Convert to Parquet (data format)
all_cs_loadcases.to_parquet('C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\all_cs_loadcases.parquet', engine = 'pyarrow', index = False)

In [2]:
all_cs_loadcases.shape

(24024000, 10)

#### Step 2: Extract minimum and maximum force values

In [3]:
# Extract unique Element names
all_elements = all_cs_loadcases['Elm'].unique()
all_elements = all_elements.tolist()

# Extract unique Load Cases
all_loadcases = all_cs_loadcases['LoadCase'].unique()
all_loadcases = all_loadcases.tolist()

In [4]:
len(all_elements)

143000

In [5]:
len(all_loadcases)

65

In [ ]:
# Define an empty list to store rows as dictionaries
result_rows = []

# Loop through elements in all_elements
for element in all_elements:

    for load_case in all_loadcases:
        # Extract element rows
        relevant_rows = all_cs_loadcases[(all_cs_loadcases['Elm'] == element) & (all_cs_loadcases['LoadCase'] == load_case)]

        # Collect min and max values for each column
        result_row = {
            'Elm': element,
            'LoadCase': load_case,
            'min_P': relevant_rows['P'].min(),
            'max_P': relevant_rows['P'].max(),
            'min_V2': relevant_rows['V2'].min(),
            'max_V2': relevant_rows['V2'].max(),
            'min_V3': relevant_rows['V3'].min(),
            'max_V3': relevant_rows['V3'].max(),
            'min_T': relevant_rows['T'].min(),
            'max_T': relevant_rows['T'].max(),
            'min_M2': relevant_rows['M2'].min(),
            'max_M2': relevant_rows['M2'].max(),
            'min_M3': relevant_rows['M3'].min(),
            'max_M3': relevant_rows['M3'].max(),
        }

        # Append the dictionary to the list
        result_rows.append(result_row)

# Create the result DataFrame
result_df = pd.DataFrame(result_rows)

In [20]:
result_df.shape

(143000, 14)

#### Step 3: Extract all frames/points from Frame combined

In [12]:
# Convert Frame combined file to DataFrame

all_frame_path = 'C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\Location\\FRAME - Combined.xlsx'
required_cols = ['Frame', 'CentroidX', 'CentroidY']

frames = pd.read_excel(all_frame_path, usecols = required_cols)
frames = frames.rename(columns = {'Frame' : 'Elm'})

#### Step 4: Merge result_df and frames @ 'Elm' 

In [17]:
# Merging frames and result_df
all_elements_df = pd.merge(result_df, frames, on = 'Elm', how = 'left')

# Reduce demicals point to a hundredth
cols_to_reduce = ['min_P', 'max_P', 'min_V2', 'max_V2', 'min_V3', 'max_V3', 'min_T', 'max_T', 'min_M2', 'max_M2', 'min_M3', 'max_M3']

# Reduce decimal points to hundredth place
all_elements_df[cols_to_reduce] = all_elements_df[cols_to_reduce].round(2)

#### Step 5: Pickling all_elements_df as Parquet/CSV

In [18]:
# Convert to Parquet (data format)
all_elements_df.to_parquet('C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\all_elements.parquet', engine = 'pyarrow', index = False)

In [19]:
# Convert to CSV (data format)
all_elements_df.to_csv('C:\\Users\\rimjhims\\Walter P. Moore and Associates\\S17-24001-00 DB Texas Instruments LFAB2 - CheeseSlab\\Cheese Slab Export\\all_elements.csv', index = False)